In [ ]:
import requests
import base64
from twilio.rest import Client
import os
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

# Microsoft Teams API credentials
teams_client_id = "<your-client-id>"
teams_client_secret = "<your-client-secret>"
teams_scope = "https://api.botframework.com/.default"
teams_token_url = "https://login.microsoftonline.com/botframework.com/oauth2/v2.0/token"
teams_webhook_url = "<your-webhook-url>"

# Twilio API credentials
twilio_account_sid = "<your-account-sid>"
twilio_auth_token = "<your-auth-token>"
twilio_from_number = "<your-twilio-number>"
twilio_to_number = "<ChatGPT-phone-number>"

# Gmail API credentials
gmail_credentials = None
gmail_token_path = "<path-to-gmail-token-file>"
gmail_credentials_path = "<path-to-gmail-credentials-file>"
gmail_from_email = "<your-gmail-email>"
gmail_to_email = "<ChatGPT-email>"


def get_teams_access_token():
    # Get an access token from the Microsoft Teams API
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {
        "client_id": teams_client_id,
        "client_secret": teams_client_secret,
        "scope": teams_scope,
        "grant_type": "client_credentials",
    }
    response = requests.post(teams_token_url, headers=headers, data=data)
    access_token = response.json().get("access_token")
    return access_token


def handle_teams_call(data):
    # Handle an incoming call from Microsoft Teams
    access_token = get_teams_access_token()
    headers = {"Authorization": "Bearer " + access_token}
    response = requests.post(teams_webhook_url, headers=headers, json=data)
    return response


def handle_gmail_email(data):
    # Handle an incoming email from Gmail
    subject = data["subject"]
    body = data["body"]
    message = f"Subject: {subject}\n\n{body}"
    client = Client(twilio_account_sid, twilio_auth_token)
    message = client.messages.create(
        body=message, from_=twilio_from_number, to=twilio_to_number
    )
    return message


def get_gmail_credentials():
    # Get credentials for the Gmail API
    credentials = None
    if os.path.exists(gmail_token_path):
        with open(gmail_token_path, "rb") as token:
            credentials = Credentials.from_authorized_user_info(
                info=base64.urlsafe_b64encode(token.read()).decode("utf-8")
            )
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                gmail_credentials_path, ["https://www.googleapis.com/auth/gmail.readonly"]
            )
            credentials = flow.run_local_server(port=0)
        with open(gmail_token_path, "wb") as token:
            token.write(base64.urlsafe_b64encode(credentials.to_json().encode())))
    return credentials


def get_gmail_email():
    # Get the most recent email from Gmail
    credentials = get_gmail_credentials()
    service = build("gmail", "v1", credentials=credentials)
    result = (
        service.users()
        .messages()
        .list(userId="me", q="is:unread from:" + gmail_from_email)
        .execute()        
    messages = result.get("messages", [])
    if messages:
        msg = messages[0]
        message = service.users().messages().get(userId="me", id=msg["id"]).execute()
        subject = ""
        body = ""
        for header in message["payload"]["headers"]:
            if header["name"] == "Subject":
                subject = header["value"]
            if header["name"] == "From":
                sender = header["value"]
        if "parts" in message["payload"]:
            parts = message["payload"]["parts"]
            data = parts[0]["body"].get("data", "")
        else:
            data = message["payload"]["body"]["data"]
        data = data.replace("-", "+").replace("_", "/")
        body = base64.b64decode(data).decode("utf-8")
        email_data = {"subject": subject, "body": body}
        handle_gmail_email(email_data)


if __name__ == "__main__":
    # Start a Flask app to handle incoming calls from Microsoft Teams
    from flask import Flask, request

    app = Flask(__name__)

    @app.route("/", methods=["POST"])
    def handle_teams_webhook():
        data = request.json
        handle_teams_call(data)
        return "OK"

    app.run(port=80, host="0.0.0.0")

    # Check for incoming emails from Gmail every minute
    import time

    while True:
        get_gmail_email()
        time.sleep(60)
       